In [48]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed=42



import torch
import torch.nn as nn
import torch.nn.functional as F
if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim


from gdeep.decision_boundary.decision_boundary_calculator import *
from gdeep.create_nets.sample_nn import SampleCNN_MNIST_SAMPLE,\
    SampleCNN_MNIST_SAMPLE_2

np.random.seed(seed)

In [82]:
batch_size = 64

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

# Filters for labels 3 and 7
trainset_1 = torch.utils.data.Subset(trainset, torch.stack((trainset.targets == 3, trainset.targets == 7), axis=-1).any(-1).nonzero()[:,0])

trainloader = torch.utils.data.DataLoader(trainset_1, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
# Filters for labels 3 and 7
testset_1 = torch.utils.data.Subset(testset, torch.stack((testset.targets == 3, testset.targets == 7), axis=-1).any(-1).nonzero()[:,0])
testloader = torch.utils.data.DataLoader(testset_1, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [80]:
#idx_3_7_train
torch.stack((trainset.targets == 3, trainset.targets == 7), axis=-1).any(-1).nonzero()[:,0]

tensor([    7,    10,    12,  ..., 59980, 59988, 59996])

In [3]:
""" # Create Dataset
threes = (path_dataset/'train/3').ls().sorted()
sevens = (path_dataset/'train/3').ls().sorted()

three_tensors = torch.stack([tensor(Image.open(o)) for o in threes]).float()/255
seven_tensors = torch.stack([tensor(Image.open(o)) for o in sevens]).float()/255

X_train = torch.cat((three_tensors, seven_tensors), 0)
X_train = X_train.unsqueeze(1) # additional color channel
y_train = torch.cat((torch.zeros(three_tensors.shape[0]), torch.ones(three_tensors.shape[0])), 0).long() """

In [43]:
digit_detect_nn= SampleCNN_MNIST_SAMPLE()

print("Trainable parameters of NN:",\
    sum(p.numel() for p in digit_detect_nn.parameters()), '\n')

print(digit_detect_nn)

Trainable parameters of NN: 61326 

SampleCNN_MNIST_SAMPLE(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [24]:
""" # Training
n_epochs = 10
batch_size = 32

loss_fkt = nn.CrossEntropyLoss()
optimizer = optim.SGD(digit_detect_nn.parameters(), lr=0.1)


for epoch in range(n_epochs):
    #digit_detect_nn.train()
    permutation = torch.randperm(X_train.size()[0])
    for i in range(0,X_train.size()[0]-batch_size, batch_size):
        optimizer.zero_grad()
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        outputs = digit_detect_nn(batch_x)
        loss = loss_fkt(outputs, batch_y)
        loss.backward()
        
        optimizer.step()
    #digit_detect_nn.eval()
    predict = digit_detect_nn(X_train)
    loss = loss_fkt(predict, y_train)
    max_idx = predict.max(dim = -1)[1]
    accuracy = (max_idx == y_train).sum()/predict.shape[0]
    print('epoch: {:3d}, loss: {:.2}, accuracy: {:.2%}'.format(epoch, float(loss), float(accuracy)))
    print(float(list(digit_detect_nn.parameters())[0][0][0][0][0])) """

epoch:   0, loss: 0.69, accuracy: 50.00%
-0.11286617815494537
epoch:   1, loss: 0.69, accuracy: 50.00%
-0.1128661185503006
epoch:   2, loss: 0.69, accuracy: 50.00%
-0.11286576837301254
epoch:   3, loss: 0.69, accuracy: 50.00%
-0.11286567151546478
epoch:   4, loss: 0.69, accuracy: 50.00%
-0.11286568641662598
epoch:   5, loss: 0.69, accuracy: 50.00%
-0.112865149974823
epoch:   6, loss: 0.69, accuracy: 50.00%
-0.11286497116088867
epoch:   7, loss: 0.69, accuracy: 50.00%
-0.11286578327417374
epoch:   8, loss: 0.69, accuracy: 50.00%
-0.11286582797765732
epoch:   9, loss: 0.69, accuracy: 50.00%
-0.11286633461713791


In [46]:
n_epoch = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(digit_detect_nn.parameters(), lr=0.001, momentum=0.9)


for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dls[1], 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = list(data)[0], list(data)[1]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = digit_detect_nn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: 0.039


TypeError: no implementation found for 'torch.nn.functional.cross_entropy' on types that implement __torch_function__: [<class 'fastai.torch_core.TensorImage'>, <class 'fastai.torch_core.TensorCategory'>]

In [38]:
dls[1]